In [41]:
import numpy as np
import torch as t
import torch.nn as nn
import pandas as pd
import torch.optim as optim
import sklearn.model_selection as sk

df = pd.read_csv("hf://datasets/syncora/developer-productivity-simulated-behavioral-data/Developer_Productivity_Synthetic_Syncora.csv")

In [42]:
df

,hours_coding,coffee_intake_mg,distractions,sleep_hours,commits,bugs_reported,ai_usage_hours,cognitive_load,task_success
0,5.190476,575,6,7.467749,6,1,0.729743,3.666136,1
1,3.069862,320,1,6.483703,0,2,0.086857,3.449533,0
2,5.052262,532,3,9.687784,9,1,0.514440,1.509259,1
3,3.694441,359,4,8.150044,3,0,1.226923,4.099178,0
4,6.883442,601,4,8.902919,5,0,0.661380,3.784728,1
...,...,...,...,...,...,...,...,...,...
1995,5.799024,532,2,4.096030,4,1,3.130257,5.657285,1
1996,5.160282,532,5,4.978048,2,3,2.046475,9.207588,0
1997,7.435158,597,6,5.687562,6,1,2.134184,7.344275,0
1998,4.918377,402,6,6.747191,4,0,1.315910,6.314791,1


In [43]:
X=df.iloc[:,:-1].to_numpy(dtype=np.float32)
Y=df.iloc[:,-1].to_numpy(dtype=np.float32)

In [44]:
df


,hours_coding,coffee_intake_mg,distractions,sleep_hours,commits,bugs_reported,ai_usage_hours,cognitive_load,task_success
0,5.190476,575,6,7.467749,6,1,0.729743,3.666136,1
1,3.069862,320,1,6.483703,0,2,0.086857,3.449533,0
2,5.052262,532,3,9.687784,9,1,0.514440,1.509259,1
3,3.694441,359,4,8.150044,3,0,1.226923,4.099178,0
4,6.883442,601,4,8.902919,5,0,0.661380,3.784728,1
...,...,...,...,...,...,...,...,...,...
1995,5.799024,532,2,4.096030,4,1,3.130257,5.657285,1
1996,5.160282,532,5,4.978048,2,3,2.046475,9.207588,0
1997,7.435158,597,6,5.687562,6,1,2.134184,7.344275,0
1998,4.918377,402,6,6.747191,4,0,1.315910,6.314791,1


In [45]:
Y=t.from_numpy(Y)
X=t.from_numpy(X)

In [46]:
X.dtype
Y=Y.reshape(-1,1)
print(Y.size(),X.size())

torch.Size([2000, 1]) torch.Size([2000, 8])


In [47]:
X = (X - X.mean(axis=0)) / X.std(axis=0)
X,X_test,Y,Y_test=sk.train_test_split(X,Y,test_size=0.30,random_state=42)

In [48]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(8, 1) 

    def forward(self, x):
        return t.sigmoid(self.linear(x))  


In [49]:
model = LogisticRegression()
criterion = nn.BCELoss() 
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [50]:
print(Y.size(),X.size())

torch.Size([1400, 1]) torch.Size([1400, 8])


In [51]:
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass
    y_pred = model(X)
    loss = criterion(y_pred, Y)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        with t.no_grad():
            y_class = (y_pred >= 0.5).float()
            acc = (y_class.eq(Y).sum().item() / Y.shape[0]) * 100
        print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {loss.item():.4f} | Accuracy: {acc:.2f}%")


Epoch [100/10000] | Loss: 0.5579 | Accuracy: 81.36%
Epoch [200/10000] | Loss: 0.4640 | Accuracy: 85.07%
Epoch [300/10000] | Loss: 0.4157 | Accuracy: 86.43%
Epoch [400/10000] | Loss: 0.3860 | Accuracy: 87.43%
Epoch [500/10000] | Loss: 0.3654 | Accuracy: 87.57%
Epoch [600/10000] | Loss: 0.3502 | Accuracy: 87.36%
Epoch [700/10000] | Loss: 0.3384 | Accuracy: 87.29%
Epoch [800/10000] | Loss: 0.3288 | Accuracy: 87.36%
Epoch [900/10000] | Loss: 0.3209 | Accuracy: 87.57%
Epoch [1000/10000] | Loss: 0.3143 | Accuracy: 87.71%
Epoch [1100/10000] | Loss: 0.3086 | Accuracy: 88.07%
Epoch [1200/10000] | Loss: 0.3036 | Accuracy: 88.29%
Epoch [1300/10000] | Loss: 0.2993 | Accuracy: 88.29%
Epoch [1400/10000] | Loss: 0.2954 | Accuracy: 88.43%
Epoch [1500/10000] | Loss: 0.2920 | Accuracy: 88.36%
Epoch [1600/10000] | Loss: 0.2890 | Accuracy: 88.21%
Epoch [1700/10000] | Loss: 0.2862 | Accuracy: 88.14%
Epoch [1800/10000] | Loss: 0.2837 | Accuracy: 88.21%
Epoch [1900/10000] | Loss: 0.2814 | Accuracy: 88.29%
Ep

In [54]:
with t.no_grad():
    y_pred=model(X_test)
    y_class = (y_pred >= 0.5).float()
    acc = (y_class.eq(Y_test).sum().item() / Y_test.shape[0]) * 100
    print(f"Accuracy: {acc:.2f}%")

Accuracy: 87.83%
